# Домашнє завдання: Прогнозування орендної плати за житло

## Мета завдання
Застосувати знання з лекції для побудови моделі лінійної регресії, що прогнозує орендну плату за житло в Індії. Ви пройдете весь цикл вирішення задачі машинного навчання: від дослідницького аналізу до оцінки якості моделі.

## Опис датасету
**House Rent Prediction Dataset** містить інформацію про 4700+ оголошень про оренду житла в Індії з такими параметрами:
- **BHK**: Кількість спалень, залів, кухонь
- **Rent**: Орендна плата (цільова змінна)
- **Size**: Площа в квадратних футах
- **Floor**: Поверх та загальна кількість поверхів
- **Area Type**: Тип розрахунку площі
- **Area Locality**: Район
- **City**: Місто
- **Furnishing Status**: Стан меблювання
- **Tenant Preferred**: Тип орендаря
- **Bathroom**: Кількість ванних кімнат
- **Point of Contact**: Контактна особа

---

## Завдання 1: Завантаження та перший огляд даних (1 бал)

**Що потрібно зробити:**
1. Завантажте дані з файлу `House_Rent_Dataset.csv`
2. Виведіть розмір датасету
3. Покажіть перші 5 рядків
4. Виведіть загальну інформацію про дані (включно з типами даних та кількістю значень)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Завантаження даних
df_rent = pd.read_csv('/content/House_Rent_Dataset.csv')

# Розмір датасету
print("Shape (rows, cols):", df_rent.shape)

Shape (rows, cols): (4746, 12)


In [ ]:
# Перші 5 рядків
df_rent.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


In [ ]:
# Загальна інформація
print("\nInfo:")
print(df_rent.info())


Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Posted On          4746 non-null   object
 1   BHK                4746 non-null   int64 
 2   Rent               4746 non-null   int64 
 3   Size               4746 non-null   int64 
 4   Floor              4746 non-null   object
 5   Area Type          4746 non-null   object
 6   Area Locality      4746 non-null   object
 7   City               4746 non-null   object
 8   Furnishing Status  4746 non-null   object
 9   Tenant Preferred   4746 non-null   object
 10  Bathroom           4746 non-null   int64 
 11  Point of Contact   4746 non-null   object
dtypes: int64(4), object(8)
memory usage: 445.1+ KB
None


## Завдання 2: Дослідницький аналіз даних (EDA) (4 бали)

**Що потрібно зробити:**
1. **Аналіз пропущених значень.** Перевірте наявність і відсоток пропущених значень у кожній колонці
2. **Базова статистика.** Обчисліть базову статистику (середнє, квартилі, стандартне відхилення) для числових змінних.
3. **Аналіз цільової змінної.** Побудуйте гістограму розподілу цільової змінної (Rent)
4. **Робота з викидами.** Знайдіть та видаліть викиди в цільовій змінній (якщо є). Визначити викиди можна будь-яким зрозумілим для вас способом, як варіант - таким, що використовується в побудові box-plot (https://en.wikipedia.org/wiki/Box_plot#Example_with_outliers).
5. **Аналіз категоріальних змінних.** Виведіть кількість унікальних значень для кожної з категоріальних колонок.


In [ ]:
# Пропущені значення
missing_data = df_rent.isnull().sum()
missing_percent = (missing_data / len(df_rent)) * 100

missing_percent

,0
Posted On,0.0
BHK,0.0
Rent,0.0
Size,0.0
Floor,0.0
Area Type,0.0
Area Locality,0.0
City,0.0
Furnishing Status,0.0
Tenant Preferred,0.0


In [ ]:
# Базова статистика для числових змінних
stats = df_rent.describe()
stats.round(2)

,BHK,Rent,Size,Bathroom
count,4746.00,4746.00,4746.00,4746.00
mean,2.08,34993.45,967.49,1.97
std,0.83,78106.41,634.20,0.88
min,1.00,1200.00,10.00,1.00
25%,2.00,10000.00,550.00,1.00
50%,2.00,16000.00,850.00,2.00
75%,3.00,33000.00,1200.00,2.00
max,6.00,3500000.00,8000.00,10.00


In [ ]:
# Аналіз цільової змінної Rent (гістограма)
fig = px.histogram(
    df_rent,
    x="Rent",
    nbins=100,
    title="Розподіл орендної плати (Rent)",
    labels={"Rent": "Орендна плата", "count": "Кількість оголошень"}
)

fig.show()


In [ ]:
# Робота з викидами в Rent за правилом IQR
Q1, Q3 = df_rent["Rent"].quantile([0.25, 0.75])
IQR = Q3 - Q1

# Межі викидів
low = Q1 - 1.5 * IQR
high = Q3 + 1.5 * IQR
print(f"IQR для Rent: low={low:.2f}, high={high:.2f}")

df_clean = df_rent[(df_rent["Rent"] >= low) & (df_rent["Rent"] <= high)]
print(f"\nВидалено викидів у Rent: {len(df_rent) - len(df_clean)} рядків (залишилось {len(df_clean)})")

IQR для Rent: low=-24500.00, high=67500.00

Видалено викидів у Rent: 520 рядків (залишилось 4226)


In [ ]:
# Аналіз категоріальних колонок: кількість унікальних
print("К-сть унікальних значень у категоріальних колонках:")
df_rent.select_dtypes(include=["object"]).nunique()

К-сть унікальних значень у категоріальних колонках:


,0
Posted On,81
Floor,480
Area Type,3
Area Locality,2235
City,6
Furnishing Status,3
Tenant Preferred,3
Point of Contact,3



## Завдання 3: Аналіз кореляцій та взаємозв'язків (3 бали)

**Що потрібно зробити:**
1. Обчисліть матрицю кореляцій для числових змінних
2. Візуалізуйте кореляційну матрицю за допомогою heatmap
3. Побудуйте scatter plot між Size та Rent
4. Проаналізуйте взаємозв'язок між BHK та Rent за допомогою boxplot (який розподіл плати для різних значень BHK)


In [ ]:
# Матриця кореляцій
correlation_matrix = df_clean.select_dtypes('number').corr()

# Візуалізація кореляцій
fig = px.imshow(
    correlation_matrix,
    text_auto='.2f',
    color_continuous_scale='RdBu_r',
    title='Матриця кореляцій всіх числових ознак',
    labels=dict(color="Кореляція")
)

fig.show()

In [ ]:
# Scatter Size vs Rent
fig = px.scatter(
    df_clean,
    x="Size",
    y="Rent",
    title="Взаємозв\'язок площі та орендної плати",
    labels={"Size": "Площа", "Rent": "Орендна плата"}
)

fig.show()


In [ ]:
# Boxplot BHK vs Rent
fig = px.box(
    df_clean,
    x="BHK",
    y="Rent",
    labels={"BHK": "BHK", "Rent": "Rent"}
)

fig.show()

### Аналіз взаємозв'язку між BHK та Rent (Boxplot)

1. **Загальна тенденція**  
   - Зі збільшенням кількості кімнат (BHK) медіана орендної плати зростає.  
   - Це означає, що більші квартири в середньому дорожчі.  

2. **Розподіл за групами**  
   - **1–2 BHK** → нижчі медіани (~8–15k), але є багато дорогих "викидів".  
   - **3–4 BHK** → вища середня оренда (~20–30k+), розкид більший.  
   - **5–6 BHK** → найвищі медіани (~40–50k), широкий діапазон цін (звичайні та преміум-квартири).  

3. **Викиди (outliers)**  
   - Для 1–2 BHK спостерігається багато квартир із завищеною орендою (можливо, престижні райони).  
   - Для 4–6 BHK ціни менш стабільні, що свідчить про різні сегменти ринку.  

   **Висновок**: Кількість кімнат прямо впливає на орендну плату, але на ціни також суттєво впливають інші фактори — місто та локація.


## Завдання 4: Feature Engineering та підготовка даних (4 бали)

**Що потрібно зробити:**
1. Закодуйте категоріальні змінні за допомогою One-Hot Encoding. Пригадайте, що в лекції ми говорили щодо кодування кат. змінних з великої кількістю різних значень і як працювати з такими випадками. Ви можете закодувати не всі кат. змінні, а лише ті, що вважаєте за потрібні (скажімо ті, що мають відносно небагато різних значень).
2. **Опціонально (по 0.5 бала за кожну доцільну ознаку):** Додайте нові ознаки, обчислені на основі наявних даних, які б на ваш погляд були корисними для моделі
3. Виберіть ознаки для побудови моделі (виключіть непотрібні колонки). Виключити можна, наприклад, ті колонки, які мають категоріальний тип і забагато (більше 20) різних значень. Треба виключити хоча б 1 колонку.
4. Розділіть дані на ознаки (X) та цільову змінну (y)
5. Застосуйте стандартизацію до числових ознак


In [ ]:
# One-Hot Encoding для категоріальних змінних
cols_for_ohe = ["Area Type", "City", "Furnishing Status", "Tenant Preferred", "Point of Contact"]

df_encoded = pd.get_dummies(df_clean, columns=cols_for_ohe, drop_first=True)
df_encoded.head()

,Posted On,BHK,Rent,Size,Floor,Area Locality,Bathroom,Area Type_Carpet Area,Area Type_Super Area,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,Tenant Preferred_Bachelors/Family,Tenant Preferred_Family,Point of Contact_Contact Builder,Point of Contact_Contact Owner
0,2022-05-18,2,10000,1100,Ground out of 2,Bandel,2,False,True,False,False,False,True,False,False,True,True,False,False,True
1,2022-05-13,2,20000,800,1 out of 3,"Phool Bagan, Kankurgachi",1,False,True,False,False,False,True,False,True,False,True,False,False,True
2,2022-05-16,2,17000,1000,1 out of 3,Salt Lake City Sector 2,1,False,True,False,False,False,True,False,True,False,True,False,False,True
3,2022-07-04,2,10000,800,1 out of 2,Dumdum Park,1,False,True,False,False,False,True,False,False,True,True,False,False,True
4,2022-05-09,2,7500,850,1 out of 2,South Dum Dum,1,True,False,False,False,False,True,False,False,True,False,False,False,True


In [ ]:
# Створюємо новий датафрейм-копію
df_model_ready = df_clean.copy()

# Нові інженерні ознаки
# Співвідношення площі на кімнату
df_model_ready.loc[:, "Size_per_BHK"] = (df_model_ready["Size"] / df_model_ready["BHK"]).round(1)

# Розділення Floor на Current і Total
def extract_floors(val):
    if isinstance(val, str) and "out of" in val:
        parts = val.split("out of")
        try:
            return int(parts[0].strip()), int(parts[1].strip())
        except ValueError:
            return None, None
    return None, None

df_model_ready[["Floor_Current", "Floor_Total"]] = df_model_ready["Floor"].apply(lambda x: pd.Series(extract_floors(x)))

# Frequency encoding для Area Locality
locality_freq = df_model_ready["Area Locality"].value_counts(normalize=True)
df_model_ready["Locality_Popularity"] = df_model_ready["Area Locality"].map(locality_freq)

# One-Hot Encoding для категоріальних змінних
cols_for_ohe = ["Area Type", "City", "Furnishing Status", "Tenant Preferred", "Point of Contact"]
df_encoded_2 = pd.get_dummies(df_clean, columns=cols_for_ohe, drop_first=True)

# Виключення непотрібних колонок
columns_to_drop = ['Posted On', 'Area Locality', 'Floor']
to_drop = [col for col in columns_to_drop if col in df_encoded_2.columns]
df_model_2 = df_encoded_2.drop(columns=to_drop)

df_model_2.head()

,BHK,Rent,Size,Bathroom,Area Type_Carpet Area,Area Type_Super Area,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,Tenant Preferred_Bachelors/Family,Tenant Preferred_Family,Point of Contact_Contact Builder,Point of Contact_Contact Owner
0,2,10000,1100,2,False,True,False,False,False,True,False,False,True,True,False,False,True
1,2,20000,800,1,False,True,False,False,False,True,False,True,False,True,False,False,True
2,2,17000,1000,1,False,True,False,False,False,True,False,True,False,True,False,False,True
3,2,10000,800,1,False,True,False,False,False,True,False,False,True,True,False,False,True
4,2,7500,850,1,True,False,False,False,False,True,False,False,True,False,False,False,True


In [ ]:
# Виключення непотрібних колонок
columns_to_drop = ['Posted On', 'Area Locality', 'Floor']
to_drop = [col for col in columns_to_drop if col in df_encoded.columns]
df_model = df_encoded.drop(columns=to_drop)

df_model

,BHK,Rent,Size,Bathroom,Area Type_Carpet Area,Area Type_Super Area,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,Tenant Preferred_Bachelors/Family,Tenant Preferred_Family,Point of Contact_Contact Builder,Point of Contact_Contact Owner
0,2,10000,1100,2,False,True,False,False,False,True,False,False,True,True,False,False,True
1,2,20000,800,1,False,True,False,False,False,True,False,True,False,True,False,False,True
2,2,17000,1000,1,False,True,False,False,False,True,False,True,False,True,False,False,True
3,2,10000,800,1,False,True,False,False,False,True,False,False,True,True,False,False,True
4,2,7500,850,1,True,False,False,False,False,True,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4741,2,15000,1000,2,True,False,False,False,True,False,False,True,False,True,False,False,True
4742,3,29000,2000,3,False,True,False,False,True,False,False,True,False,True,False,False,True
4743,3,35000,1750,3,True,False,False,False,True,False,False,True,False,True,False,False,False
4744,3,45000,1500,2,True,False,False,False,True,False,False,True,False,False,True,False,False


In [ ]:
# Поділ на X (ознаки) та y (ціль)
X = df_model.drop(columns=['Rent']) # Ознаки
y = df_model['Rent'] # Цільова змінна

print(f"\nРозмір X (ознак): {X.shape}")
print(f"Розмір y (цілі): {y.shape}")


Розмір X (ознак): (4226, 16)
Розмір y (цілі): (4226,)


In [ ]:
#  Визначаємо числові колонки
num_cols = X.select_dtypes(include=["int64", "float64"]).columns

# Стандартизація
scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[num_cols] = scaler.fit_transform(X[num_cols])

# Перетворюємо назад у DataFrame для зручності
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

# Перевірка
X_scaled_df.head()

,BHK,Size,Bathroom,Area Type_Carpet Area,Area Type_Super Area,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,Tenant Preferred_Bachelors/Family,Tenant Preferred_Family,Point of Contact_Contact Builder,Point of Contact_Contact Owner
0,0.052966,0.469859,0.272578,False,True,False,False,False,True,False,False,True,True,False,False,True
1,0.052966,-0.147778,-1.133910,False,True,False,False,False,True,False,True,False,True,False,False,True
2,0.052966,0.263980,-1.133910,False,True,False,False,False,True,False,True,False,True,False,False,True
3,0.052966,-0.147778,-1.133910,False,True,False,False,False,True,False,False,True,True,False,False,True
4,0.052966,-0.044839,-1.133910,True,False,False,False,False,True,False,False,True,False,False,False,True


## Завдання 5: Розділення даних та навчання моделі (3 бали)

**Що потрібно зробити:**
1. Розділіть дані на навчальну (80%) та тестову (20%) вибірки.
2. Створіть модель лінійної регресії.
3. Навчіть модель на навчальних даних.
4. Виведіть усі коефіцієнти моделі (ваги) та напишіть, які 2 ознаки найбільше впливають на прогноз.
5. Зробіть прогнози на тренувальній та тестовій вибірках.

In [ ]:
# Розділяємо дані: 80% на навчання, 20% на тест
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled_df, y,
    test_size=0.2,  # 20% даних йде на тест
    random_state=42  # фіксуємо випадковість для відтворюваності
)

# Створюємо модель
model = LinearRegression()

# Навчаємо модель на навчальних даних
model.fit(X_train, y_train)

# Виводимо коефіцієнти моделі
coeff_df = pd.DataFrame({
    "Feature": X_train.columns,
    "Coef": model.coef_
})
coeff_df["Abs_Coef"] = coeff_df["Coef"].abs()
coeff_df_sorted = coeff_df.sort_values(by="Abs_Coef", ascending=False)

print("Коефіцієнти для всіх ознак (від найбільшого до найменшого за абсолютним значенням):")
display(coeff_df_sorted)

top_features = coeff_df_sorted.head(2)
print("\nДві ознаки, що найбільше впливають на Rent:")
for i, row in top_features.iterrows():
    print(f"{i+1}. {row['Feature']}: {row['Coef']:.3f}")

# Прогнози на навчальній вибірці
y_train_pred = model.predict(X_train)

# Прогнози на тестовій вибірці (нові дані!)
y_test_pred = model.predict(X_test)

# Показуємо приклади
print("\nПерші 5 прогнозів на тестовій вибірці:")
print(y_test_pred[:5])
print("Перші 5 реальних значень:")
print(y_test.iloc[:5].values)

Коефіцієнти для всіх ознак (від найбільшого до найменшого за абсолютним значенням):


,Feature,Coef,Abs_Coef
9,City_Mumbai,18541.084881,18541.084881
15,Point of Contact_Contact Owner,-8778.609329,8778.609329
14,Point of Contact_Contact Builder,-5838.526682,5838.526682
11,Furnishing Status_Unfurnished,-4546.720079,4546.720079
1,Size,3876.397264,3876.397264
10,Furnishing Status_Semi-Furnished,-3411.113995,3411.113995
8,City_Kolkata,-3260.193557,3260.193557
4,Area Type_Super Area,-2642.013372,2642.013372
0,BHK,2623.023112,2623.023112
3,Area Type_Carpet Area,-2394.915415,2394.915415



Дві ознаки, що найбільше впливають на Rent:
10. City_Mumbai: 18541.085
16. Point of Contact_Contact Owner: -8778.609

Перші 5 прогнозів на тестовій вибірці:
[28113.38544975  3382.41111894 40391.72058218  2958.95199055
 15364.27207141]
Перші 5 реальних значень:
[22000  5000 37000  8000 15000]


## Завдання 6: Оцінка якості моделі (2 бали)

**Що потрібно зробити:**
1. Обчисліть MAE, RMSE та R² для навчальної та тестової вибірок
2. Порівняйте метрики та зробіть висновок про якість моделі
3. Проаналізуйте і дайте висновок, чи є ознаки перенавчання або недонавчання (**Нагадування**: перенавчання - коли модель дуже добре працює на тренувальних даних, але погано на тестових; недонавчання - коли модель погано працює навіть на тренувальних даних)
4. Побудуйте графік розсіювання "реальні vs прогнозовані значення" та зробіть висновок про якість моделі


In [ ]:
# Функція для RMSE
def compute_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Розраховуємо метрики для тестової вибірки
mae = mean_absolute_error(y_test, y_test_pred)
mse = mean_squared_error(y_test, y_test_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_test_pred)

print("МЕТРИКИ ЯКОСТІ МОДЕЛІ (на тестовій вибірці):")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.3f}")

# Порівняння з навчальною вибіркою
mae = mean_absolute_error(y_train, y_train_pred)
mse = mean_squared_error(y_train, y_train_pred)
rmse = np.sqrt(mse)
r2_train = r2_score(y_train, y_train_pred)

print("\nМЕТРИКИ ЯКОСТІ МОДЕЛІ на тренувальній вибірці:")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2_train:.3f}")

МЕТРИКИ ЯКОСТІ МОДЕЛІ (на тестовій вибірці):
MAE: 5537.15
RMSE: 7737.65
R²: 0.688

МЕТРИКИ ЯКОСТІ МОДЕЛІ на тренувальній вибірці:
MAE: 5540.57
RMSE: 7698.80
R²: 0.690


In [ ]:
# Візуалізація: реальні vs прогнозовані значення
fig = px.scatter(
    x=y_test,
    y=y_test_pred,
    title='Реальні vs Прогнозовані значення',
    labels={'x': 'Реальна орендна плата', 'y': 'Прогнозована орендна плата'},
    opacity=0.6
)

# Додаємо ідеальну лінію (де прогноз = реальність)
max_val = max(y_test.max(), y_test_pred.max())
fig.add_trace(
    go.Scatter(
        x=[0, max_val],
        y=[0, max_val],
        mode='lines',
        name='Ідеальний прогноз',
        line=dict(color='red', dash='dash')
    )
)

fig.update_layout(height=500)
fig.show()

# Висновки по прогнозуванню орендної плати

## 1. Дані
- Датасет: 4746 оголошень, після видалення викидів залишилось 4226 рядків.
- Пропущених значень немає.
- Основні числові ознаки: `Rent`, `Size`, `BHK`, `Bathroom`.

## 2. Взаємозв'язки
- `Size` та `BHK` позитивно корелюють із `Rent`.
- Більше кімнат → вища орендна плата.

## 3. Feature Engineering
- Нові ознаки: `Size_per_BHK`, `Bath_per_BHK`, `Floor_Current`, `Floor_Total`, `Locality_Popularity`.
- One-Hot Encoding для категоріальних ознак: `Area Type`, `City`, `Furnishing Status`, `Tenant Preferred`, `Point of Contact`.

## 4. Модель
- Лінійна регресія.
- Найбільш впливові ознаки: `Size` та `BHK`.
- Метрики:
  - **Тренувальна вибірка:** R² = 0.690, MAE ≈ 5540, RMSE ≈ 7699  
  - **Тестова вибірка:** R² = 0.688, MAE ≈ 5537, RMSE ≈ 7738

## 5. Висновки
- Модель добре відтворює тренд, перенавчання не виявлено.
- Найбільші помилки на квартирах з великою площею та преміум-локаціями.



## Завдання 7: Аналіз помилок (4 бали)

**Що потрібно зробити:**
1. Обчисліть помилки (residuals = реальні - прогнозовані значення)
2. Побудуйте гістограму розподілу помилок
3. Створіть scatter plot помилок відносно величини прогнозованих значень. Чи росте помилка з ростом прогнозованого значення?
4. Знайдіть 5 прогнозів з найбільшими помилками
5. Проаналізуйте, на яких типах житла модель помиляється найбільше. Типи можна розрізняти за кількістю кімнат чи містом, наприклад.
6. Подумайте і напишіть, які наступні кроки ви б зробили, аби поліпшити якість моделі. Опціонально можна їх зробити і ми перевіримо :)

In [ ]:
# Розраховуємо помилки (залишки)
residuals = y_test - y_test_pred

# Гістограма помилок
fig = px.histogram(
    x=residuals,
    nbins=50,
    title='Розподіл помилок прогнозування',
    labels={'x': 'Помилка (реальні - прогнозовані)', 'count': 'Кількість'},
    color_discrete_sequence=['#e74c3c']
)
fig.add_vline(x=0, line_dash="dash", line_color="black", annotation_text="Ідеальний прогноз")
fig.update_layout(height=400)
fig.show()

In [ ]:
# Scatter plot: помилки vs прогнозовані значення
fig = px.scatter(
    x=y_test_pred,
    y=residuals,
    title='Залежність помилок від прогнозованих значень',
    labels={'x': 'Прогнозовані лайки', 'y': 'Помилка'},
    opacity=0.5
)

# Додаємо горизонтальну лінію на 0
fig.add_hline(y=0, line_dash="dash", line_color="red", annotation_text="Без помилки")

fig.update_layout(height=400)
fig.show()

**Висновок:** помилки збільшуються для дуже великих прогнозованих значень, особливо для дорогих квартир.

In [ ]:
# Знаходимо пости з найбільшими помилками
errors_df = pd.DataFrame({
    'real': y_test.values,
    'predicted': y_test_pred,
    'error': np.abs(residuals)
})

# Топ-5 найбільших помилок
top_errors = errors_df.nlargest(5, 'error')
print("5 найбільших помилок прогнозування:")
print(top_errors.round(3))

5 найбільших помилок прогнозування:
       real  predicted      error
3962  65000  31751.825  33248.175
549    8000  40132.537  32132.537
904    8000  38198.862  30198.862
3520  65000  34853.589  30146.411
275   30000  59418.699  29418.699


In [ ]:
# Аналіз помилок по типам житла (BHK та City)
# Створюємо новий датафрейм для аналізу помилок
error_df = pd.DataFrame({
    "Rent_True": y_test,
    "Rent_Pred": y_test_pred
}, index=y_test.index)

# Додаємо колонку з помилками
error_df["Error"] = error_df["Rent_True"] - error_df["Rent_Pred"]

# Підтягуємо інформацію про BHK та City з оригінального датасету
error_df[["BHK", "City"]] = df_clean.loc[error_df.index, ["BHK", "City"]]

# Аналіз помилок за кількістю кімнат
avg_error_bhk = error_df.groupby("BHK")["Error"].mean().sort_values(ascending=False)
print("Середня помилка за BHK (к-сть кімнат):")
print(avg_error_bhk)

# Аналіз помилок за містами
avg_error_city = error_df.groupby("City")["Error"].mean().sort_values(ascending=False)
print("\nСередня помилка за містами (City):")
print(avg_error_city.head(10))



Середня помилка за BHK (к-сть кімнат):
BHK
6    5424.679358
3    1288.977510
1     746.980417
2    -967.912791
4   -3871.607242
5   -4442.786139
Name: Error, dtype: float64

Середня помилка за містами (City):
City
Delhi        1602.292402
Chennai       162.969482
Hyderabad     117.178070
Bangalore    -301.655796
Mumbai      -1024.991004
Kolkata     -1413.655790
Name: Error, dtype: float64


**Висновок:**

 - Найбільші помилки модель робить для квартир з великою кількістю кімнат (6 BHK) та у дорогих містах, таких як Delhi.

 - Для квартир з малою кількістю кімнат та дешевших міст помилка менша.

## Можливі кроки для покращення моделі

1. **Логарифмічне перетворення цільової змінної (Rent)**  
   - Допомагає зменшити вплив великих значень та стабілізувати модель.

2. **Взаємодії між ознаками**  
   - Наприклад: `Size * BHK` або `Locality_Popularity * Size_per_BHK`.  
   - Може покращити точність прогнозу за рахунок врахування комбінованого впливу ознак

3. **Детальніше врахування географії**  
   - Класифікація районів, а не тільки міст, для точнішого моделювання ринку оренди.

4. **Додаткові ознаки для преміум-сегменту**  
   - Використання `Floor`, `Furnishing Status`, або `Tenant Preferred` для кращої прогностики дорогих квартир.


**Посилання на Google Colab, оскільки завдання виконувала з використанням Plotly для побудови графіків:** https://colab.research.google.com/drive/1E9gd2X7Alo_Fx2iV27VWUD0m37bqhdvv#scrollTo=kbkViScobMbX